# Ancestral Sequences and the Phylogenetic Tree
Program to prune the phylogenetic tree file of all species to only those modern species that would be used for ASR. Looks at the nodes the ancestral sequences will occupy to keep a track of them. 

Milo Thordarson (anth2886@student.uu.se)

## Setup

In [2]:
from ete3 import Tree
from Bio import SeqIO
import pandas

# Variables that will need changing
# We need the SNP file, as it has the annotations of which sequences are Ancestral
tree_file_name = "treefile_pette"
snp_file_name = "snpAlignment_Spyrou2022_modern_ancient.fasta2"
outgroup = "outgroup_Y.pseudo"

## Defining Functions

In [3]:
def get_ancient_species_list(name_file, tree):
    """Function to obatain a list of all the ancient species
    NOTE: this will be highly dependent on the kind of data and the naming scheme, hence why for example there is a hard coded naming change in this function, 
    so be sure to change this function to suit the needs of the current dataset. Returns a list of the species names."""
    # Get ids from snp file that are ancient, removing the last 8 characters that are the tag
    ancient_species_list = [x.id[:-8] for x in name_file if "Ancient" in x.id]
    print(f'Number of ancient samples to start: {len(ancient_species_list)}')

    # Get names of all species from the treefile
    tree_names = list(tree.iter_leaf_names())
    
    # For loop to go through and make sure that the ancient names from the snp file match those in the tree exactly
    for i in range(len(ancient_species_list)):
        if ancient_species_list[i] not in tree_names:
            # Finding the name as the prefix matches
            name = next((name for name in tree_names if ancient_species_list[i] in name), None)
            if name is not None:
                ancient_species_list[i] = name
            # Well, the prefix matches, except for this known exception, so I am doing it manually
            elif ancient_species_list[i] == "COL1":
                ancient_species_list[i] = "COLC1-COLC2a_COLC2b"
    return ancient_species_list


def informative_nodes(tree):
  """Function to give names to the internal nodes so they can be searched for and stored as ancestral nodes informativelty. Returns the tree with the annotations."""
  # Nodes will be named from 0, starting at the root
  count = 0
  for node in tree.traverse():
    if not node.is_leaf():
      node.name = "intrnl" + str(count)
      count += 1
  return tree

def informative_nodes_numbers(tree):
  """Function to give names to the internal nodes so they can be searched for and stored as ancestral nodes informativelty. Returns the tree with the annotations."""
  # Nodes will be named from 0, starting at the root
  count = 0
  for node in tree.traverse():
    if not node.is_leaf():
      node.name = str(count)
      count += 1
  return tree


def create_annotation_dataframe(ancient_sample_names, tree):
    """Function to create a dataframe with information about the ancient samples including name, branch lenghts of that leaf, the parent node that the sequence represents, 
    and whether that node has any of the modern sister or ancient sister issues (see section Problems below). Returns a pandas dataframe with all the information."""
    # Creating the pandas dataframe that will be the output
    d = {'name': ancient_sample_names}
    df = pandas.DataFrame(data = d)

    # List to keep a track of the branch length of each ancient sample
    distances = []
    # Seting up a count for the number of times an issue is found and a list so they can be coded into the dataframe
    ancient_sisters = []
    sisters = []
    # Also keeps a track of the parent nodes for the dataframe
    parents = []

    for node_name in ancient_sample_names:
        distances.append(tree.search_nodes(name=node_name)[0].dist)
        # Set up check for problems, and store parent names
        ancient_problem = False
        sister_problem = False  
        node = tree.search_nodes(name=node_name)[0]
        parent = node.up
        for child in parent.get_children():
            # Make sure you don't count the ancient sample we're already looking at
            if child.is_leaf() and child.name != node_name: 
                parents.append(parent.name)
                if child.name in ancient_sample_names: 
                    ancient_problem = True
                    ancient_sisters.append(child.name)
                else:
                    sister_problem = True
                    sisters.append(child.name)
            elif child.name != node_name: parents.append(child.name)
        # If no problems were found, code that with a dash
        if ancient_problem == False: ancient_sisters.append("-")
        if sister_problem == False: sisters.append("-")

    df['dist'] = distances
    df['parent'] = parents
    df['anct_problem'] = ancient_sisters
    df['sis_problem'] = sisters
    
    return df


def annotate_tree(ancient_sample_names, tree):
    """Function to add the attribute anct to the nodes in the tree that have ancient samples as leaf taxa. Returns the tree with the annotations added."""
    for node_name in ancient_sample_names:
        node = tree.search_nodes(name=node_name)[0]
        parent = node.up
        for child in parent.get_children():
            # Make sure you don't count the ancient sample we're already looking at
            if child.is_leaf() and child.name != node_name: 
                grandparent = parent.up
                grandparent.add_features(anct=True)
            else: parent.add_features(anct=True)
    return tree


def create_ancient_dictionary(ancient_sample_names, tree):
    """Function to create a dictionary allowing the user to search for specific ancestral leaves and obtain the section of the tree they might represent. Returns this dictionary."""
    ancient_dictionary = {}
    for node_name in ancient_sample_names:
        node = tree.search_nodes(name=node_name)[0]
        parent = node.up
        for child in parent.get_children():
            # Make sure you don't count the ancient sample we're already looking at
            if child.is_leaf() and child.name != node_name: 
                grandparent = parent.up
                ancient_dictionary[node_name] = grandparent
            else: ancient_dictionary[node_name] = parent
    return ancient_dictionary


def prune_ancient_samples_no_nodes(ancient_sample_names, tree):
    """Function that prunes the ancient samples from the tree. Returns the pruned tree."""
    tree_names = list(tree.iter_leaf_names())
    # Prune the tree, agrument taken is species to retain
    tree.prune([x for x in tree_names if x not in ancient_sample_names], preserve_branch_length=True)
    return tree


def prune_ancient_samples_keep_nodes(ancient_sample_names, tree):
    """Function that prunes the ancient samples from the tree, but it keeps all the same nodes in the tree. Returns the pruned tree."""
    all_nodes = list(tree.iter_search_nodes())
    all_names = [node.name for node in all_nodes]
    # Prune the tree, agrument taken is species to retain
    tree.prune([x for x in all_names if x not in ancient_sample_names], preserve_branch_length=True)
    return tree

## Problems
The interpretation of the ASR sequences will be different if the ancient sequence we are looking at is a direct sister group to any taxa: that means that it is not an ancestor at a node, but along the evolution of the mdoern branch and therefore will not be reconstructed. But if the sister taxa is another ancient sequence, then ASR will reconstruct the ancestor to both those sequences and the modern, meaning that interpretation is harder. 

## Running the code

In [4]:
# First, read in the treefile as a Tree in ete3 and snp file as a list from a fasta
full_tree = Tree(tree_file_name)
snp_fasta = list(SeqIO.parse(open(snp_file_name), 'fasta'))

# Then root the tree using the outgroup
full_tree.set_outgroup(outgroup)

print(f'Number of leaves in tree file: {len(full_tree)}')
print(f'Number of leaves in snp file: {len(snp_fasta)}')

Number of leaves in tree file: 251
Number of leaves in snp file: 251


In [5]:
ancient = get_ancient_species_list(snp_fasta, full_tree)

# Double check that we are left with a list of the same length of ancient samples, and that all the names are found in the treefile
print(f'Number of ancient samples: {len(ancient)}, all ancient names found in tree names: {all(x in list(full_tree.iter_leaf_names()) for x in ancient)}')

Number of ancient samples to start: 47
Number of ancient samples: 47, all ancient names found in tree names: True


In [6]:

# Get informative names of the nodes
full_tree_node_names = informative_nodes(full_tree)
full_tree_annotated = annotate_tree(ancient, full_tree_node_names)

anct_dict = create_ancient_dictionary(ancient, full_tree_node_names)
print(anct_dict[ancient[1]].get_ascii(show_internal = True))


         /-Azov38
        |
        |                  /-Rostov2033
        |                 |
        |                 |         /-CHE.C1-C2-3-C4
-intrnl194        /intrnl211       |
        |        |        |        |                                    /-OBS124
        |        |        |        |                           /intrnl246
        |        |         \intrnl219                 /intrnl242        \-OBS116
        |        |                 |                 |        |
         \intrnl201                |         /intrnl236        \-OBS110
                 |                 |        |        |
                 |                  \intrnl227        \-OBS137
                 |                          |
                 |                           \-OBS107
                 |
                  \-PEB10


In [7]:

# Displaying it sorted by the distance, so we can see which ancient sequences are closest to being the real ancestral sequence. 
ancient_df = create_annotation_dataframe(ancient, full_tree_annotated)
display(ancient_df.sort_values(by=['dist'], ascending=True))
#display(ancient_df)

,name,dist,parent,anct_problem,sis_problem
42,OBS124,0.000002,intrnl246,OBS116,-
0,BSK001-003.A0101.A0102.A0103-malt,0.000002,intrnl78,-,-
39,London_EastSmithfield_8124_8291_11972,0.000002,intrnl145,-,-
36,STN019.A0101,0.000002,intrnl237,-,-
35,STN008.A0101,0.000002,intrnl228,-,-
34,STN021.A0101,0.000002,intrnl247,STN013.A0101,-
33,STN020.A0101,0.000002,intrnl247,-,-
32,STN014.A0101,0.000002,intrnl213,-,-
31,STN013.A0101,0.000002,intrnl247,STN021.A0101,-
30,STN007.A0101,0.000002,intrnl220,-,-


In [8]:
# remove the counting from the for loops, do it here. 
#print(f'Number of ancient sister problems = {ancient_problem_count}')
#print(f'Number of sister problems = {sister_problem_count}')
#print(f'Total problematic ancient samples = {ancient_problem_count + sister_problem_count}')
#print(f'Total useful ancient samples = {len(ancient) - (ancient_problem_count + sister_problem_count)}')

In [9]:
#pruned_tree = prune_ancient_samples_no_nodes(ancient, full_tree_annotated)


# Then create a pruned tree file in newick format
#pruned_tree.write(format=1, outfile="pruned_tree_outgroup_dist.treefile")

In [11]:
# This is to look for each ancient sample if they have any modern ancestors in the tree
for name in ancient:
    print("_________________________________________________________")
    print(name)
    print(anct_dict[name])
    print()

_________________________________________________________
BSK001-003.A0101.A0102.A0103-malt

                                       /- /- /- /- /- /- /- /-intrnl246
                                 /- /-|
                              /-|      \- /- /-intrnl212
                             |  |
                           /-|   \- /-intrnl189
                          |  |
                          |  |            /-intrnl203
                  /- /- /-|   \- /- /- /-|
                 |        |               \- /- /- /- /- /- /-intrnl247
               /-|        |
              |  |         \-intrnl174
            /-|  |
           |  |   \-intrnl144
           |  |
           |   \- /- /- /-intrnl169
           |
           |   /- /-intrnl146
           |  |
           |  |         /-1.ANT1_UG05-0454
           |  |      /-|
           |  |     |   \-1.ANT1_Antiqua
           |  |     |
           |  |     |                                 /-1.ORI1_CA88
           |  |     |         

In [12]:
# This is the pruning protocol, some things are commented out when files could be overwritten
full_tree = Tree(tree_file_name)
full_tree.set_outgroup(outgroup)
full_tree_node_names = informative_nodes_numbers(full_tree)

subtree = full_tree_node_names.search_nodes(name="78")[0].up

useful = ["BSK001-003.A0101.A0102.A0103-malt", "Bolgar"]

not_useful = [name for name in ancient if name not in useful]

remove_useless_ancient = prune_ancient_samples_no_nodes(not_useful, subtree)

pruned_subtree = prune_ancient_samples_keep_nodes(useful, subtree)

print(pruned_subtree)

#leaves = list(pruned_subtree.iter_leaf_names())

#print(leaves)

#subtree.write(format=1, outfile="subtree_no_nodes_numbers.treefile")

#with open("species_list_subtree.txt", "w") as f:
          #f.write("\n".join(map(str, leaves)))


               /-3.ANT1a_7b
              |
            /-|      /-3.ANT1b_CMCC71001
           |  |   /-|
           |   \-|   \-3.ANT1c_C1976001
           |     |
           |      \-3.ANT1d_71021
         /-|
        |  |         /-3.ANT2a_MGJZ6
        |  |      /-|
        |  |   /-|   \-3.ANT2b_MGJZ7
        |  |  |  |
        |   \-|   \-3.ANT2e_MGJZ3
        |     |
        |     |   /-3.ANT2c_MGJZ9
        |      \-|
      /-|         \-3.ANT2d_MGJZ11
     |  |
     |  |         /-4.ANT1a_MGJZ12
     |  |      /-|
     |  |     |  |   /-4.ANT_1454
     |  |     |   \-|
     |  |   /-|     |   /-4.ANT_338
     |  |  |  |      \-|
     |  |  |  |         \-4.ANT_517
     |   \-|  |
     |     |   \-4.ANT_M-1944
     |     |
     |      \-4.ANT_KM932
     |
     |         /-1.ANT1_UG05-0454
     |      /-|
     |     |   \-1.ANT1_Antiqua
     |     |
     |     |                                 /-1.ORI1_CA88
     |     |                              /-|
     |     |            

In [13]:
# This was to create the tree with the Ancient tag on the ancient samples to see them better in figtree
full_tree = Tree(tree_file_name)
full_tree.set_outgroup(outgroup)
full_tree_node_names = informative_nodes(full_tree)

for node in full_tree_node_names.iter_leaves():
    if node.name in ancient: node.name += "_ANCIENT"

full_tree_node_names.write(format=1, outfile="whole_tree.treefile")